The `tangle` and `transform_cell` modules are non-pidgin source.

In [1]:
    import ast, IPython, re, functools,traitlets, doctest, textwrap, mistune, types, importnb, typing as t, pygments
    if __name__ == '__main__': 
        %reload_ext pidgin

In [2]:
    def markdown_to_python(str: "Markdown Source") -> "Valid Python Source": 
        return BlockLexer().parse(''.join(str))

In [3]:
    @functools.wraps(markdown_to_python)
    def markdown_to_python_wrapper(line: str, cell: str): 
        return IPython.display.HTML(pygments.highlight(markdown_to_python(cell), pygments.lexers.PythonLexer(), pygments.formatters.HtmlFormatter(noclasses=True)))

In [4]:
    def load_ipython_extension(shell):
        unload_ipython_extension(shell)
        shell.input_transformer_manager.cleanup_transforms.insert(0, Tangle(parent=shell))
        for i, transformer in enumerate(shell.input_transformers_cleanup):
            try:
                if transformer.initial_re.pattern[1:4] == '>>>':
                    shell.input_transformers_cleanup.pop(i)
                    break
            except: ...
             
        try: IPython.core.magic.register_cell_magic(markdown_to_python_wrapper)
        except: shell.log.error("Unable to load the pidgin.tangle cell magic.")

In [5]:
    class Tangle(traitlets.config.LoggingConfigurable): 
        markdown = traitlets.Bool(True, help="""Convert __Markdown__ source to Python & execute the block code.""")
        log_level = traitlets.Int(10)
        def __call__(self, lines: t.List[str]) -> t.List[str]:
            if self.markdown:
                source = markdown_to_python(lines + ['\n']).splitlines(True)
                if source:
                    self.parent.log.log(self.log_level, F"In[{len(self.parent.user_ns['In'])}]:")
                    self.parent.log.log(self.log_level, ''.join(source))
                    return source
            return lines

In [6]:
    class BlockLexer(mistune.BlockLexer):
        raw = unindented = indented = None
        def __init__(BlockLexer, *args, **kwargs):
            super().__init__(*args, **kwargs)
            BlockLexer.original = None
            BlockLexer.raw, BlockLexer.unindented, BlockLexer.indented, BlockLexer.min_indent = [], [], [], 0
            
        def parse_block_code(BlockLexer, m=None):
            if m:
                if get_line_indent(get_last_line(BlockLexer.raw)) == get_line_indent(m.string[slice(*m.span())]):
                    return BlockLexer.parse_generic(m)
            
            BlockLexer.format()
            BlockLexer.raw.extend(m and BlockLexer.pop_text(m) or [])
            if m: super().parse_block_code(m)
            BlockLexer.indent()

            
        def parse(self, text: str, rules=None) -> t.List[t.Dict]: 
            if text.startswith('%%'): return text
            
            text = text.lstrip('//')
            if any((self.original, self.raw, self.unindented, self.indented)): 
                return super().parse(text, rules)
            
            text = strip_blank_lines(text)
            self.original = ''.join(text).splitlines(True)
            
            super().parse(text, rules) # returns tokens but we dont need them.
            
            while self.original:  self.raw.append(self.original.pop(0))
            self.min_indent = self.min_indent or 4
            self.format(punc=';')
            self.indent()
            return textwrap.dedent(''.join(self.indented))
                    
        def parse_generic(self, m) -> str: 
            self.raw.extend(self.pop_text(m))
            
        parse_doctest = parse_block_html = parse_block_quote = parse_fences =\
        parse_heading = parse_hrule = parse_lheading = parse_newline =\
        parse_nptable = parse_paragraph = parse_table = parse_text =  parse_generic

        def parse_def_footnotes(self, m):
            self.format()
            text = ''.join(self.pop_text(m))
            key, sep, body = text.lstrip('[').lstrip('^').partition(']:')
            key = quote(key)
            quoted = quote(body.lstrip())
            self.unindented.extend(F"""globals()[{key}] = {quoted if quoted.strip() else "None"}""".splitlines(True))
            
        def parse_def_links(self, m):
            self.parse_block_code()
            text = ''.join(self.pop_text(m))
            key, sep, body = text.lstrip('[').lstrip('^').partition(']:')
            key = quote(key)

            self.unindented.extend(F"""globals()['__context__'] = globals().get('__context__', dict()); globals()['__context__'][{key}] = {quote(body.lstrip())}""".splitlines(True))

In [7]:
    def clear_original(BlockLexer) -> None:
        while BlockLexer.original and not BlockLexer.original[0].strip(): 
            BlockLexer.raw += [BlockLexer.original.pop(0)]
        
    def pop_text(BlockLexer, m) -> str:
        text = []
        lines = m.string[slice(*m.span())].splitlines(True)
        # Remove empty lines
        while lines and not lines[0].strip(): lines.pop(0)

        # Drop the lines from the original body
        if BlockLexer.original:
            while lines:
                line = lines.pop(0)
                if line.strip():
                    while line.strip() not in BlockLexer.original[0]: 
                        text += [BlockLexer.original.pop(0)]
                    text += [BlockLexer.original.pop(0)]
        return text

    def format(BlockLexer, punc: "Punctation on the quoted code"=''):
        BlockLexer.clear_original()            
        if BlockLexer.raw: 
            last_line = get_last_line(BlockLexer.indented).rstrip()
            lines = [line for line in BlockLexer.raw]
            if not last_line.endswith(('"""', "'''", "---", "'''\\", '"""\\',)):
                lines = [quote(lines, punc)]
            BlockLexer.unindented.extend(lines)
            BlockLexer.raw = []


    def indent(BlockLexer):
        # Extract the first line of the current code block.
        first_line = get_first_line(BlockLexer.raw)
        # Construct the code we'll 
        code, body = ''.join(BlockLexer.raw), ''.join(BlockLexer.unindented)

        # The previous last line append
        last_line = get_last_line(BlockLexer.indented)

        # The current indent level so far.
        prior_indent = get_line_indent(last_line)

        # Does the last line enter a block statement
        definition = last_line.rstrip().endswith(':')
        returns = _has_return(BlockLexer.indented)

        this_indent = get_line_indent(get_first_line(BlockLexer.raw))


        # Assign the minimum indent 
        if not BlockLexer.min_indent: 
            BlockLexer.min_indent = this_indent



        if this_indent < BlockLexer.min_indent:
            code = textwrap.indent(code, ' '*(BlockLexer.min_indent-this_indent))
            this_indent = get_line_indent(get_first_line(code.splitlines()))


        # Normalize the indent we'll assign the body+code
        indent = max(BlockLexer.min_indent, (returns and min or max)(prior_indent, this_indent))        

        if definition:
            if prior_indent >= indent:
                indent = (prior_indent + 4)

            body = hanging_indent(textwrap.indent(body, ' '*BlockLexer.min_indent), ' '*(indent-BlockLexer.min_indent))
        else:
            body = textwrap.indent(body, ' '*indent)

        # Cell Magics
        if code.lstrip().startswith('%%'):
            # Cell magics can be split across __Markdown__ blocks.  With this 
            # approach conditional blocks can be used with magics.
            code = (' '*this_indent) + importnb.loader.dedent(code)
            # might have to add lines if line sized changed.

        if BlockLexer.min_indent:
            BlockLexer.indented.extend(body.splitlines(True) + code.splitlines(True))
            BlockLexer.unindented = []
        BlockLexer.raw = []
        return ''       

    
    BlockLexer.indent = indent
    BlockLexer.format = format
    BlockLexer.clear_original = clear_original
    BlockLexer.pop_text = pop_text

In [8]:
    def unload_ipython_extension(shell):
        shell.input_transformer_manager.cleanup_transforms = [object for object in shell.input_transformer_manager.cleanup_transforms if not isinstance(object, Tangle)]


In [9]:
    def quote(str, punc=''):
        str, leading_ws = ''.join(str), []
        lines = str.splitlines(True)
        _ = '"""'
        if _ in str: _ = "'''"
        if not str.strip(): _ = punc = ''
        while lines and (not lines[0].strip()): leading_ws.append(lines.pop(0))    
        str = ''.join(lines)
        end = len(str.rstrip())
        str, ending_ws = str[:end], str[end:]
        if str and str.endswith(_[0]): str += ' '                    
        return F"{''.join(leading_ws)}{_}{str}{_}{punc}{ending_ws}"

`get_first_line` get the first non-`iter`able strings in 

In [10]:
    def get_first_line(lines, line=''):
        for line in lines or ['']: 
            if line.strip(): break
        return line
    def get_last_line(lines, line=''): return get_first_line(reversed(lines), line)

`get_line_indent` computes the indent of a string.

In [11]:
    def get_line_indent(line):  return len(line) - len(line.lstrip())

In [12]:
    def _has_return(code):
        code = '\n'.join(code)
        if 'return ' not in code: return False
        code = importnb.loader.dedent(code)
        try:
            node = ast.parse(code)
            while hasattr(node, 'body'): node = node.body[-1]
            return isinstance(node, ast.Return)
        except: ...  

In [13]:
    def hanging_indent(str, indent: str, *, out="""""") -> str:
        for line in str.splitlines(True):
            if not line.strip(): out += line
            else:
                if out.strip(): out += line
                else: out += indent+line
        return out

In [14]:
    def strip_blank_lines(str): return '\n'.join(str if str.strip() else '' for str in ''.join(str).splitlines())